# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time

import numpy as np

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    chars_per_batch = batch_size * n_steps
    n_batches = len(arr)//chars_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * chars_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:, n+1:n+n_steps+1]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [8]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [9]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


In [10]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, n_tokens, n_steps=50, n_layers=2, 
                 n_hidden=256, drop_prob=0.5):
        
        super().__init__()
        
        # Store parameters
        self.n_tokens = n_tokens
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        
        # Define layers
        self.dropout = nn.Dropout(drop_prob)
        self.lstm = nn.LSTM(self.n_tokens, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.fc = nn.Linear(n_hidden, self.n_tokens)
        
    def forward(self, x, hc):
        
        # x = input, h = hidden state, c = cell state
        x, (h, c) = self.lstm(x, hc)
        
        x = self.dropout(x)
        
        # Stack up LSTM outputs 
        batch_size = x.size()[0]
        n_steps = x.size()[1]
        x = x.view(batch_size * n_steps, self.n_hidden)
        
        x = F.log_softmax(self.fc(x), dim=1)
        
        return x, (h, c)

In [12]:
def init_hidden(net, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(net.parameters()).data
        return (Variable(weight.new(net.n_layers, batch_size, net.n_hidden).zero_()),
                Variable(weight.new(net.n_layers, batch_size, net.n_hidden).zero_()))

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [167]:
batch_size = 100        # Sequences per batch
n_steps = 100           # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
n_layers = 2            # Number of LSTM layers
learning_rate = 0.005   # Learning rate
drop_prob = 0.2         # Dropout drop probability
clip = 5                # Gradient clipping

## Time for training

In [168]:
epochs = 20
print_every = 10
cuda = True

net = CharRNN(len(vocab), n_steps=n_steps, n_layers=n_layers, 
              n_hidden=lstm_size, drop_prob=drop_prob)

opt = optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

if cuda:
    net.cuda()

counter = 0
n_chars = len(vocab)
for e in range(epochs):
    # init hc a tuple of (hidden, cell) states
    hc = init_hidden(net, batch_size)
    for x, y in get_batches(encoded, batch_size, n_steps):
        counter += 1

        # One-hot encode our data and make them Torch tensors
        x = one_hot_encode(x, n_chars)
        x, y = torch.from_numpy(x), torch.from_numpy(y)

        inputs, targets = Variable(x), Variable(y.long())
        if cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        # Creating new variables for the hidden/cell state, otherwise
        # we'd backprop through the entire training history
        hc = tuple([Variable(each.data) for each in hc])

        net.zero_grad()

        output, hc = net.forward(inputs, hc)
        loss = criterion(output, targets.view(batch_size * n_steps))
        
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm(net.parameters(), clip)

        opt.step()

        if counter % print_every == 0:
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.4f}...".format(loss.data[0]))

Epoch: 1/20... Step: 10... Loss: 3.1692...
Epoch: 1/20... Step: 20... Loss: 3.1058...
Epoch: 1/20... Step: 30... Loss: 3.1249...
Epoch: 1/20... Step: 40... Loss: 3.0916...
Epoch: 1/20... Step: 50... Loss: 3.0507...
Epoch: 1/20... Step: 60... Loss: 2.9645...
Epoch: 1/20... Step: 70... Loss: 2.8561...
Epoch: 1/20... Step: 80... Loss: 2.7512...
Epoch: 1/20... Step: 90... Loss: 2.6269...
Epoch: 1/20... Step: 100... Loss: 2.5104...
Epoch: 1/20... Step: 110... Loss: 2.4128...
Epoch: 1/20... Step: 120... Loss: 2.3846...
Epoch: 1/20... Step: 130... Loss: 2.3226...
Epoch: 1/20... Step: 140... Loss: 2.2583...
Epoch: 1/20... Step: 150... Loss: 2.2081...
Epoch: 1/20... Step: 160... Loss: 2.1868...
Epoch: 1/20... Step: 170... Loss: 2.1093...
Epoch: 1/20... Step: 180... Loss: 2.0707...
Epoch: 1/20... Step: 190... Loss: 2.0576...
Epoch: 2/20... Step: 200... Loss: 2.2204...
Epoch: 2/20... Step: 210... Loss: 2.0383...
Epoch: 2/20... Step: 220... Loss: 2.0298...
Epoch: 2/20... Step: 230... Loss: 1.9313.

Epoch: 10/20... Step: 1860... Loss: 1.0794...
Epoch: 10/20... Step: 1870... Loss: 1.0722...
Epoch: 10/20... Step: 1880... Loss: 1.0575...
Epoch: 10/20... Step: 1890... Loss: 1.0870...
Epoch: 10/20... Step: 1900... Loss: 1.0957...
Epoch: 10/20... Step: 1910... Loss: 1.0845...
Epoch: 10/20... Step: 1920... Loss: 1.1070...
Epoch: 10/20... Step: 1930... Loss: 1.1184...
Epoch: 10/20... Step: 1940... Loss: 1.0567...
Epoch: 10/20... Step: 1950... Loss: 1.1360...
Epoch: 10/20... Step: 1960... Loss: 1.1225...
Epoch: 10/20... Step: 1970... Loss: 1.0764...
Epoch: 10/20... Step: 1980... Loss: 1.1680...
Epoch: 11/20... Step: 1990... Loss: 1.0801...
Epoch: 11/20... Step: 2000... Loss: 1.1032...
Epoch: 11/20... Step: 2010... Loss: 1.1046...
Epoch: 11/20... Step: 2020... Loss: 1.0832...
Epoch: 11/20... Step: 2030... Loss: 1.0842...
Epoch: 11/20... Step: 2040... Loss: 1.1037...
Epoch: 11/20... Step: 2050... Loss: 1.0748...
Epoch: 11/20... Step: 2060... Loss: 1.0640...
Epoch: 11/20... Step: 2070... Loss

Epoch: 19/20... Step: 3650... Loss: 0.9875...
Epoch: 19/20... Step: 3660... Loss: 0.9950...
Epoch: 19/20... Step: 3670... Loss: 0.9830...
Epoch: 19/20... Step: 3680... Loss: 0.9618...
Epoch: 19/20... Step: 3690... Loss: 0.9593...
Epoch: 19/20... Step: 3700... Loss: 0.9912...
Epoch: 19/20... Step: 3710... Loss: 1.0064...
Epoch: 19/20... Step: 3720... Loss: 0.9769...
Epoch: 19/20... Step: 3730... Loss: 0.9775...
Epoch: 19/20... Step: 3740... Loss: 0.9659...
Epoch: 19/20... Step: 3750... Loss: 0.9666...
Epoch: 19/20... Step: 3760... Loss: 0.9797...
Epoch: 20/20... Step: 3770... Loss: 0.9856...
Epoch: 20/20... Step: 3780... Loss: 1.0034...
Epoch: 20/20... Step: 3790... Loss: 1.0050...
Epoch: 20/20... Step: 3800... Loss: 0.9559...
Epoch: 20/20... Step: 3810... Loss: 0.9629...
Epoch: 20/20... Step: 3820... Loss: 0.9684...
Epoch: 20/20... Step: 3830... Loss: 0.9766...
Epoch: 20/20... Step: 3840... Loss: 0.9917...
Epoch: 20/20... Step: 3850... Loss: 0.9802...
Epoch: 20/20... Step: 3860... Loss

In [16]:
torch.save(net.state_dict(), 'anna_rnn.net')

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.

In [129]:
def predict(net, char, hc=None, cuda=False):
    
    if hc is None:
        hc = init_hidden(net, 1)
    
    x = one_hot_encode(np.array([[char]]), net.n_tokens)
    
    # Make sure our variables are volatile so we don't save the history
    # since we're in inference mode here
    inputs = Variable(torch.from_numpy(x), volatile=True)
    hc = tuple([Variable(each.data, volatile=True) for each in hc])
    
    if cuda:
        inputs = inputs.cuda()

    x, hc = net.forward(inputs, hc)
    
    return x, hc

In [157]:
def choose_char(x, top_k=None):
    if top_k is None:
        ps, out = torch.exp(x).max(dim=0)
        out = out[0].data.numpy()[0]
    else:
        probs, idx = torch.exp(x).topk(top_k)
        probs, idx = probs.data.numpy().squeeze(), idx.data.numpy().squeeze()
        out = np.random.choice(idx, p=probs/probs.sum())
    return out

In [158]:
def sample(net, n_samples, prime="The", cuda=False, top_k=None):
        ''' Sample from a trained network.
        '''
        # First make sure the network is in inference mode
        net.eval()
        if cuda:
            net.cuda()
        else:
            net.cpu()
        
        # Initialize hidden state
        hc = init_hidden(net, 1)
        
        # Build up the hidden state from the priming text
        sample = list(prime)
        for char in sample:
            x, hc = predict(net, vocab_to_int[char], hc=hc, cuda=cuda)
        
        # Get the first new character
        if cuda:
            x = x.cpu()
        char_int = choose_char(x)
        sample.append(int_to_vocab[char_int])
        
        for ii in range(n_samples):
            x, hc = predict(net, char_int, hc=hc, cuda=cuda)
            if cuda:
                x = x.cpu()
            char_int = choose_char(x, top_k=top_k)
            sample.append(int_to_vocab[char_int])
            
        return sample

In [166]:
print(''.join(sample(net, 1000, top_k=5)))

The
woman struck and drew a strung of all the carriage
what he was not friendly of him, and would not criming to see him and so much
frightening what had
been seem, when
they had so straight out.

"Yes, I don't know about her," he addressed home, sat
down and say that the porter sat
down to him. "Though I shall not stand at the same
force over the
sense of the praceity of the same to you to tell me to drive or. It seems
or to be servants
with me."

"And what does he had not say?" he asked was
that her
shoulders, so that they could not be anything, and so as a minute
that she was simply talking at her there."

"Yes, there's none that things should have sumplecely attended? And I have told her husband what I works alone in, that had been
bowed to my dear, and though I won't consider the porter's side.

"Why didn't you to give it."

He set off till the stream of late and carelessness of this. Having had terrible on his face. She did anything to her a long while. He saw that she could
not 

Here, pass in the path to a checkpoint and sample from the network.